### Linear model
- Miller et al. 2021

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
import matplotlib.pyplot as plt
from neuropy import plotting
from scipy import stats


basepath = Path("D:\\Data")
# files = ["gronckle.csv", "grump.csv"]
files = sorted(basepath.glob("*.csv"))

fig = plotting.Fig(6, 3, size=(12, 5), num=1)

npast = 10
params_pooled = []
task_type_bool = []

for i, file in enumerate(files):
    data_df = pd.read_csv(basepath / file)
    prob_corr = np.abs(
        stats.pearsonr(data_df["rewprobfull1"], data_df["rewprobfull2"])[0]
    )

    task_type = "unstructured" if prob_corr < 0.2 else "structured"
    task_type_bool.append(prob_corr)

    choices = data_df["port"].to_numpy()
    choices[choices == 2] = -1
    outcomes = data_df["reward"].to_numpy()
    outcomes[outcomes == 0] = -1
    n_trials = choices.size

    past_choices = sliding_window_view(choices, npast)[:-1, :]
    past_outcomes = sliding_window_view(outcomes, npast)[:-1, :]
    actual_choices = choices[npast:]

    x = np.hstack(
        (
            past_choices * past_outcomes,
            past_choices,
            past_outcomes,
        )
    )
    clf = LogisticRegression(random_state=0).fit(x, actual_choices)

    params = np.fliplr(clf.coef_.squeeze().reshape(3, npast))
    params_pooled.append(params)

    subfig = fig.add_subfigure(fig.gs[i])
    subfig.suptitle(f"{files[i].name[:-4]}, {task_type}")
    sub_axs = subfig.subplots(1, 3, width_ratios=[1, 1, 1], sharey=True, sharex=True)

    colors = ["orange", "purple", "blue"]
    titles = ["Reward Seeking", "Choice Preservation", "Main effect of Outcome"]
    for _, ax in enumerate(sub_axs):

        ax.plot(np.arange(1, 11), params[_], ".-", color=colors[_], zorder=1)
        ax.set_title(titles[_])
        ax.axhline(0, color="gray", zorder=0, lw=0.8)
        ax.set_xticks([1, 5, 10])

    if i == 0:
        sub_axs[0].set_xlabel("Trials in the past")
        sub_axs[0].set_ylabel("Influence on current choice")

task_type_bool = np.array(task_type_bool)
params_pooled = np.array(params_pooled)
mean_struc = params_pooled[task_type_bool < 0.2, :, :].mean(axis=0)
mean_unstruc = params_pooled[task_type_bool > 0.2, :, :].mean(axis=0)

subfig = fig.add_subfigure(fig.gs[4:, 0:2])
subfig.suptitle(f"Mean across animals by task type")
sub_axs = subfig.subplots(1, 3, width_ratios=[1, 1, 1], sharey=True, sharex=True)

# colors = ["orange", "purple", "blue"]
colors = ["#5040BF", "#AFBF40"]


titles = ["Reward Seeking", "Choice Preservation", "Main effect of Outcome"]
for _, ax in enumerate(sub_axs):

    ax.plot(np.arange(1, 11), mean_struc[_], ".-", color=colors[0], alpha=0.7, zorder=1)
    ax.plot(
        np.arange(1, 11), mean_unstruc[_], ".-", color=colors[1], alpha=0.7, zorder=1
    )
    ax.legend(["Struc", "Unstruc"])
    ax.set_title(titles[_])
    ax.axhline(0, color="gray", zorder=0, lw=0.8)
    ax.set_xticks([1, 5, 10])

### Estimating Qlearning parameters
- Estimate learning parameter for chosen and unchosen arms/choices. If the animals are learning, then one should expect that the chosen and unchosen arm will have postive and negative $\alpha$ respectively.

In [ ]:
import numpy as np
from pathlib import Path
from scipy import stats
import pandas as pd
from banditpy.core import MultiArmedBandit
from banditpy.analyses import QlearningEstimator

basepath = Path("D:\\Data")
# files = ["gronckle.csv", "grump.csv"]
files = sorted(basepath.glob("*.csv"))

params_df = []

for i, file in enumerate(files):
    mab = MultiArmedBandit.from_csv(basepath / file).keep_sessions_by_trials(
        min_trials=100, clip_max=100
    )

    if mab.is_structured:
        print(f"{file.name[:-4]} is structured")
    else:
        print(f"{file.name[:-4]} is unstructured")

    qlearn = QlearningEstimator(mab)
    qlearn.fit(
        x0=None,
        bounds=np.array([(-1, 1), (-1, 1), (0.005, 20)]),
        method="diff",
        n_opts=5,
        n_cpu=4,
    )

    qlearn.print_params()
    df = pd.DataFrame(
        {
            "name": file.name[:-4],
            "param": ["alpha_chosen", "alpha_unchosen", "beta"],
            "param_values": np.array([qlearn.alpha_c, qlearn.alpha_u, qlearn.beta]),
            "grp": "struc" if mab.is_structured else "unstruc",
        }
    )
    params_df.append(df)

params_df = pd.concat(params_df, ignore_index=True)

In [ ]:
from neuropy import plotting
import pandas as pd
import seaborn as sns

fig = plotting.Fig(1, 2, size=(4, 3), num=1)
ax1 = fig.subplot(fig.gs[0])
ax2 = fig.subplot(fig.gs[1])

plot_kw = dict(x="param", y="param_values", hue="task_type")
bar_kw = dict(
    errorbar="se",
    palette="dark:black",
    linestyle="none",
    alpha=0.5,
    dodge=0.4,
    zorder=1,
    marker=".",
    markersize=10,
    markeredgewidth=0,
    err_kws={"linewidth": 1},
)
strip_kw = dict(palette="husl", alpha=0.5, dodge=True, zorder=2)


alpha_df = params_df[params_df["param"] != "Beta"]
sns.pointplot(alpha_df, ax=ax1, **plot_kw, **bar_kw)
sns.stripplot(alpha_df, ax=ax1, **plot_kw, **strip_kw)

beta_df = params_df[params_df["param"] == "Beta"]
sns.pointplot(beta_df, ax=ax2, **plot_kw, **bar_kw)
sns.stripplot(beta_df, ax=ax2, **plot_kw, **strip_kw)

ax1.set_ylabel("Estimated alpha values")
ax2.set_ylabel("Estimated beta values")
fig.fig.suptitle("Q-learning in two-armed bandit task")

Text(0.5, 0.98, 'Q-learning in two-armed bandit task')

In [ ]:
from pybads import BADS
import numpy as np


def noisy_sphere(x, sigma=1.0):
    """Simple quadratic function with added noise."""
    x_2d = np.atleast_2d(x)
    f = np.sum(x_2d**2, axis=1)
    noise = sigma * np.random.normal(size=x_2d.shape[0])
    return f + noise


x0 = np.array([-3, -3])
# Starting point
lower_bounds = np.array([-5, -5])
upper_bounds = np.array([5, 5])
plausible_lower_bounds = np.array([-2, -2])
plausible_upper_bounds = np.array([2, 2])

options = {
    "uncertainty_handling": True,
    "max_fun_evals": 300,
    "noise_final_samples": 100,
}
bads = BADS(
    noisy_sphere,
    x0,
    lower_bounds,
    upper_bounds,
    plausible_lower_bounds,
    plausible_upper_bounds,
    options=options,
)
optimize_result = bads.optimize()

In [ ]:
val = 2
exec("D" + "=val")

In [ ]:
def f(x):
    evaluation_parameters = {"D": 2}
    for key, val in evaluation_parameters.items():
        exec(key + "=val")
    m = D / 2

    return m

In [ ]:
f(3)

In [ ]:
a = {"M": 3}
for k, val in a.items():
    print(k, val)
    exec(k + "=val")
print(M)

In [ ]:
val = 2
exec("D=val")

print(D)